In [ ]:
# Función para verificar nombres de columnas
def check_column_names(symbol, output_dir):
    file_path = f"/content/drive/My Drive/tradingcripto/data/processed/{symbol}_processed.csv"
    if os.path.exists(file_path):
        df = pd.read_csv(file_path, index_col='timestamp', parse_dates=True)
        print(f"Column names for {symbol}:")
        print(df.columns)
    else:
        print(f"No data file found for {symbol}")

# Verificar nombres de columnas para una criptomoneda
symbol = 'BTCUSDT'
output_dir = "/content/drive/My Drive/tradingcripto/data/interim"
check_column_names(symbol, output_dir)

In [ ]:
# Función para cargar datos procesados
def load_processed_data(symbols, output_dir):
    data = {}
    for symbol in symbols:
        file_path = f"/content/drive/My Drive/tradingcripto/data/processed/{symbol}_processed.csv"
        if os.path.exists(file_path):
            df = pd.read_csv(file_path, index_col='timestamp', parse_dates=True)
            data[symbol] = df
        else:
            print(f"No data file found for {symbol}")
    return data

# Función para realizar EDA
def eda(df, symbol):
    print(f"--- {symbol} ---")

    # Estadísticas descriptivas
    print(df.describe())

    # Visualización de precios
    plt.figure(figsize=(14, 7))
    plt.plot(df[f'{symbol}_close'], label='Cierre')
    plt.plot(df[f'{symbol}_sma_50'], label='SMA 50')
    plt.plot(df[f'{symbol}_ema_50'], label='EMA 50')
    plt.title(f'Precio de Cierre y Medias Móviles de {symbol}')
    plt.legend()
    plt.show()

    # Histograma del volumen
    plt.figure(figsize=(14, 7))
    df[f'{symbol}_volume'].hist(bins=50)
    plt.title(f'Histograma del Volumen de {symbol}')
    plt.show()

    # Heatmap de correlaciones
    plt.figure(figsize=(14, 7))
    sns.heatmap(df.corr(), annot=True, fmt='.2f', cmap='coolwarm')
    plt.title(f'Mapa de Calor de Correlaciones de {symbol}')
    plt.show()

    # Análisis de Tendencias y Estacionalidad
    result = seasonal_decompose(df[f'{symbol}_close'], model='multiplicative', period=24)
    result.plot()
    plt.show()

# Cargar los datos procesados
symbols = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'LTCUSDT']
output_dir = "/content/drive/My Drive/tradingcripto/data/interim"
data = load_processed_data(symbols, output_dir)

# Realizar EDA para cada criptomoneda
for symbol in symbols:
    eda(data[symbol], symbol)

In [ ]:
# Función para cargar datos procesados
def load_processed_data(symbols, output_dir):
    data = {}
    for symbol in symbols:
        file_path = f"/content/drive/My Drive/tradingcripto/data/interim/{symbol}_processed.csv"
        if os.path.exists(file_path):
            df = pd.read_csv(file_path, index_col='timestamp', parse_dates=True)
            data[symbol] = df
        else:
            print(f"No data file found for {symbol}")
    return data

# Cargar los datos procesados
symbols = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'LTCUSDT']
output_dir = "/content/drive/My Drive/tradingcripto/data/interim"
data = load_processed_data(symbols, output_dir)

In [ ]:
# Verificar datos cargados
for symbol in symbols:
    print(f"Primeras filas de los datos para {symbol}:")
    display(data[symbol].head())
    print(f"Resumen de datos para {symbol}:")
    display(data[symbol].describe())
    print(f"Valores NaN en los datos para {symbol}:")
    display(data[symbol].isna().sum())

In [ ]:
# Verificar valores NaN después de la eliminación de características correlacionadas
for symbol in symbols:
    print(f"Valores NaN en los datos reducidos para {symbol}:")
    display(data[symbol].isna().sum())

    # Rellenar valores NaN con el valor más cercano
    if data[symbol].isna().sum().sum() > 0:
        print(f"Rellenando valores NaN para {symbol}")
        data[symbol].fillna(method='ffill', inplace=True)
        data[symbol].fillna(method='bfill', inplace=True)
        display(data[symbol].isna().sum())


In [ ]:
symbols = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'LTCUSDT']

def prepare_data(df, target_column, sequence_length=50):
    # Seleccionar todas las columnas excepto la columna objetivo
    X = df.drop(columns=[target_column]).values
    y = df[target_column].values

    feature_scaler = RobustScaler()
    target_scaler = RobustScaler()

    X_scaled = feature_scaler.fit_transform(X)
    y_scaled = target_scaler.fit_transform(y.reshape(-1, 1)).flatten()

    X_seq, y_seq = create_sequences(X_scaled, y_scaled, sequence_length)

    split_index = int(len(X_seq) * 0.8)
    X_train, X_test = X_seq[:split_index], X_seq[split_index:]
    y_train, y_test = y_seq[:split_index], y_seq[split_index:]

    print(f"Tamaño de X_train: {len(X_train)}, Tamaño de X_test: {len(X_test)}")
    print(f"Tamaño de y_train: {len(y_train)}, Tamaño de y_test: {len(y_test)}")

    # Guardar los datos preparados
    #prepared_data_path = os.path.join("..//content/drive/My Drive/tradingcripto/data/processed", f"{symbol}_prepared_data.pkl")
    #joblib.dump((X_train, X_test, y_train, y_test, feature_scaler, target_scaler), prepared_data_path)

    # Guardar los escaladores
    scaler_path = os.path.join("..//content/drive/My Drive/tradingcripto/data/processed", f"{symbol}_scalers.pkl")
    joblib.dump((feature_scaler, target_scaler), scaler_path)

     # Guardar un ejemplo de los datos procesados y escalados
    example_data_path = os.path.join("..//content/drive/My Drive/tradingcripto/data/processed", f"{symbol}_example_data.pkl")
    joblib.dump((X[:5], X_scaled[:5]), example_data_path)  # Guardar los primeros 5 ejemplos de datos

    return X_train, X_test, y_train, y_test, feature_scaler, target_scaler

def create_sequences(X, y, seq_length):
    xs, ys = [], []
    for i in range(len(X) - seq_length):
        xs.append(X[i:i + seq_length])
        ys.append(y[i + seq_length])
    return np.array(xs), np.array(ys)

# Función para verificar solapamiento
def check_overlap(train_data, test_data):
    train_set = set(map(tuple, train_data.reshape((train_data.shape[0], -1))))
    test_set = set(map(tuple, test_data.reshape((test_data.shape[0], -1))))
    overlap = train_set.intersection(test_set)
    return len(overlap) > 0

# Preparar datos para cada criptomoneda
sequence_length = 50
prepared_data = {}
for symbol in symbols:
    print(f"Preparando datos para {symbol}")
    target_column = f'{symbol}_next_close'
    # Utilizar todas las columnas excepto la columna objetivo
    X_train, X_test, y_train, y_test, feature_scaler, target_scaler = prepare_data(data[symbol], target_column, sequence_length)

    # Verificación de solapamiento
    if check_overlap(X_train, X_test):
        raise ValueError(f"Existe solapamiento entre los conjuntos de entrenamiento y validación para {symbol}")

    prepared_data[symbol] = (X_train, X_test, y_train, y_test, feature_scaler, target_scaler)
    prepared_data_path = os.path.join("..//content/drive/My Drive/tradingcripto/data/processed", f"{symbol}_prepared_data.pkl")
    joblib.dump((X_train, X_test, y_train, y_test, feature_scaler, target_scaler), prepared_data_path)

print("Datos preparados y guardados.")